In [1]:
import os
import numpy as np
import urduhack
from collections import Counter

c:\Users\AhmedAli\anaconda3\envs\UGEC\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\AhmedAli\anaconda3\envs\UGEC\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
I

In [2]:
data = open("urmono.txt", "r").readlines()
data = [x.strip() for x in data[:500_000]]
data = np.array(data)

In [3]:
def replace_digits(text):
    new_text = ""
    for char in text:
        if char in "0123456789":
            new_text += chr(ord(char) - ord('0') + ord('۰'))
        else:
            new_text += char
    return new_text

normalizer = np.vectorize(urduhack.normalization.normalize)
digit_normalizer = np.vectorize(replace_digits)
data = normalizer(data)
data = digit_normalizer(data)

In [4]:
letters = [char for line in data for char in line]
letters = Counter(letters)
letters = [(ord(k), k, v) for k, v in letters.items()]
letters.sort(reverse=True)
allowed = set()
with open("allowed.txt", "w") as f:
    f.write("[")
    for k, c, v in letters:
        if v > 100 and k not in list(range(33, 123)):
            allowed.add(c)
            f.write(f"{k}, ")
    f.write("]\n")

In [9]:
clean_text = lambda text: "".join([c for c in text if c in allowed]).strip()
data = np.array([clean_text(x) for x in data if len(clean_text(x)) > 50 and len(clean_text(x)) < 100])
data = np.unique(data)
np.random.shuffle(data)
len(data)

140574

In [10]:
os.makedirs("cleaned_correct_corpus", exist_ok=True)
for i in range(0, len(data), 50_000):
    with open(f"cleaned_correct_corpus/data_{i//50_000:02d}.txt", "w") as f:
        f.write("\n".join(data[i:i+50_000]))

In [11]:
!winrar a -r cleaned_correct_corpus.rar cleaned_correct_corpus